## Funtions to use

>* Connecting to SQL DB
>* Data transformations
>* PLotting to generate an HTML dash
>* Email sending code
>* Error handling

Scheduling the script to be run every day. 

#### things to show as visualization


    Today's counts. 
    Counts by flow types 
    counts by shift
    Successful proceeses in the past getting failed count
    count by project
    Comparisions of today with week, month
    Average Elapse time, average 

#### Data transformations and charts 
>* create a function to take , now time , time difference to take and returns only required data
>* create a function that takes now time and no of days(n) and gets summary data of past n days
>* create a function to make charts

In [1]:
import numpy as np 
import pandas as pd
import datetime

import plotly
# print(plotly.__version__)  # version >1.9.4 required
from plotly.graph_objs import Scatter, Layout

In [2]:
df = pd.read_csv("Failed_Logs_kohls.csv")
df.head()

,execution_id,Flow,Project,user,Start_Time,End_Time,Elapsed,Job_Name,status,error_Reason,Flow_Type,Shift,ETL_Load_Time,Issue_Type,Last_Successful_Run
0,1080708,atg_top_offers_executor_flow,Marketing,pzhdusr,05-06-2018 5.03,05-06-2018 5.13,10m 0s,atg_top_offers_executer_flowtocheck,Failed,Last execution of the flow Data_Acquisition:kl...,High,AfterNoon,2018-06-05 18:31:45.311556,Application,NaN
1,1080708,atg_top_offers_executor_flow,Marketing,pzhdusr,05-06-2018 5.03,05-06-2018 5.13,10m 0s,atg_top_offers_executer_flowtocheck,Failed,"Not started.null', Last execution of the flow ...",High,AfterNoon,2018-06-05 18:36:49.989653,Application,NaN
2,1080879,nativeapp_denorm_flow,Nativeapp,pzhdusr,05-06-2018 8.45,05-06-2018 9.01,15m 0s,nativeapp_check_cba_flow,Failed,Last execution of the flow Clickstream:cart_an...,Critical,AfterNoon,2018-06-05 21:44:25.237108,Application,NaN
3,1080874,atg_top_offers_executor_flow,Marketing,pzhdusr,05-06-2018 8.42,05-06-2018 8.52,10m 0s,atg_top_offers_executer_flowtocheck,Failed,"Not started.null', Last execution of the flow ...",High,AfterNoon,2018-06-05 21:44:25.237108,Application,NaN
4,1080863,dmnd-sales-current-cncld-ord-flow,BigData-ECOM,pzhdusr,05-06-2018 8.36,05-06-2018 8.36,4 sec,dmnd-sales-current-cncld-ord-data-transformation,Failed,' No new records to process because DMND_SLS_C...,High,AfterNoon,2018-06-05 21:44:25.237108,Application,NaN


In [3]:
df.columns

Index(['execution_id', 'Flow', 'Project', 'user', 'Start_Time', 'End_Time',
       'Elapsed', 'Job_Name', 'status', 'error_Reason', 'Flow_Type', 'Shift',
       'ETL_Load_Time', 'Issue_Type', 'Last_Successful_Run'],
      dtype='object')

In [4]:
vars = ['Flow', 'Project', 'Job_Name','Flow_Type', 'Shift', 'Issue_Type']

In [5]:
df[vars[5]].value_counts()

Application    923
Environment     53
Name: Issue_Type, dtype: int64

In [6]:
def convertSQLDateTimeToTimestamp(value):
    return datetime.datetime.strptime(value, '%m-%d-%Y %H.%M')

df['Start_Time'] = df['Start_Time'].apply(lambda x : convertSQLDateTimeToTimestamp(x))

In [7]:
plotly.offline.plot({
"data": [
    Scatter(x=[1, 2, 3, 4], y=[4, 1, 3, 7])
],
"layout": Layout(
    title="hello world"
)
})

'file://C:\\Users\\Admin\\WHISK SOFTWARE PVT. LTD\\Macys - POC\\temp-plot.html'

In [8]:
# The public plotly graphs to include in the email. These can also be generated with `py.plot(figure, filename)`
graphs = [
    'https://plot.ly/~christopherp/308',
    'https://plot.ly/~christopherp/306',
    'https://plot.ly/~christopherp/300',
    'https://plot.ly/~christopherp/296'
]


from IPython.display import display, HTML

template = (''
    '<a href="{graph_url}" target="_blank">' # Open the interactive graph when you click on the image
        '<img src="{graph_url}.png">'        # Use the ".png" magic url so that the latest, most-up-to-date image is included
    '</a>'
    '{caption}'                              # Optional caption to include below the graph
    '<br>'                                   # Line break
    '<hr>'                                   # horizontal line
'')

email_body = ''
for graph in graphs:
    _ = template
    _ = _.format(graph_url=graph, caption='')
    email_body += _

display(HTML(email_body))

In [9]:
me  = 'lavanyadav009@gmail.com'
recipient = 'lavanya.davluri@gspann.com'
subject = 'Graph Report'

email_server_host = 'smtp.gmail.com'
port = 587
email_username = me
email_password = '2VD08ec009'


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os

msg = MIMEMultipart('alternative')
msg['From'] = me
msg['To'] = recipient
msg['Subject'] = subject

msg.attach(MIMEText(email_body, 'html'))

server = smtplib.SMTP(email_server_host, port)
server.ehlo()
server.starttls()
server.login(email_username, email_password)
server.sendmail(me, recipient, msg.as_string())
server.close()